In [1]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
import speech_recognition as sr



In [2]:
# used a dictionary to represent an intents JSON file
with open("../training_data/intents.json", "r", encoding='utf-8') as f:
    data = json.load(f)
    
print(data)

{'intents': [{'tag': 'greetings', 'patterns': ['Hi', 'Hey', 'Hello', 'How are you?', 'Good day'], 'responses': ["Hello, I'm the University Chatbot"]}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'closing', 'patterns': ['Bye', 'Goodbye'], 'responses': ['Goodbye']}, {'tag': 'email', 'patterns': ['What is the email address of Finance Office?', 'Finance Office email address', 'Email address Finance Office'], 'responses': ['The email address of the Finance Office is<br><a>finance.ar@slu.edu.ph</a> <br> For online payments please email your reciept on this email: <br> <a>claims.payment.tuition@slu.edu.ph</a> <br> <a>finance.ar@slu.edu.ph</a><br> <a>claimspayment@slu.edu.ph</a>']}, {'tag': 'landline', 'patterns': ['What are the SLU landline numbers?', 'SLU landline numbers'], 'responses': ['+(63) (74) 442 3043; 442 2793; 442 2193; 443 2001; 444 8246 to 48 (loc. 279)']}, {'tag':

In [3]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes = []
doc_X = []
doc_y = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))
print(words)

["'s", 'a', 'about', 'ac', 'accountancy', 'address', 'am', 'an', 'and', 'applicant', 'application', 'applying', 'arch', 'archi', 'architecture', 'are', 'art', 'available', 'b', 'ba', 'beced', 'beed', 'bio', 'biology', 'bpe', 'bsarchi', 'bsba', 'bsba-em', 'bsba-hrdm', 'bsce', 'bsche', 'bscs', 'bsed', 'bsee', 'bsem', 'bsge', 'bshm', 'bsie', 'bsit', 'bsme', 'bsmls', 'bspolsci', 'bsswk', 'by', 'bye', 'can', 'checklist', 'com', 'computing', 'contact', 'could', 'course', 'curiculim', 'curiculum', 'currently', 'day', 'dean', 'description', 'do', 'education', 'email', 'eng', 'engineer', 'engineering', 'engl', 'enroll', 'enrolled', 'enrollment', 'entrance', 'entrep', 'exam', 'examination', 'existing', 'failed', 'fee', 'finance', 'finman', 'for', 'freshies', 'freshman', 'funny', 'good', 'goodbye', 'grade', 'have', 'hello', 'helpful', 'hey', 'hi', 'how', 'i', 'if', 'in', 'incoming', 'industrial', 'information', 'is', 'it', 'joke', 'know', 'landline', 'law', 'liberal', 'lot', 'ma', 'management', '

In [4]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [5]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               23168     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 38)                2470      
Total params: 33,894
Trainable params: 33,894
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
6/6 [==============================] - 0s 2ms/step - loss: 3.5475 - accuracy: 0.0899
Epoch 2/200
6/6 

6/6 [==============================] - 0s 1ms/step - loss: 0.1516 - accuracy: 0.9382
Epoch 75/200
6/6 [==============================] - 0s 1ms/step - loss: 0.2011 - accuracy: 0.9270
Epoch 76/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1760 - accuracy: 0.9494
Epoch 77/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1963 - accuracy: 0.9101
Epoch 78/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1616 - accuracy: 0.9382
Epoch 79/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1086 - accuracy: 0.9438
Epoch 80/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1066 - accuracy: 0.9663
Epoch 81/200
6/6 [==============================] - 0s 2ms/step - loss: 0.1305 - accuracy: 0.9607
Epoch 82/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1193 - accuracy: 0.9551
Epoch 83/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1286 - accuracy: 0.9438
Epoch 84/200
6/6 [===============

6/6 [==============================] - 0s 1ms/step - loss: 0.2043 - accuracy: 0.9213
Epoch 158/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1005 - accuracy: 0.9551
Epoch 159/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1735 - accuracy: 0.9438
Epoch 160/200
6/6 [==============================] - 0s 2ms/step - loss: 0.2254 - accuracy: 0.9494
Epoch 161/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1287 - accuracy: 0.9607
Epoch 162/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1346 - accuracy: 0.9494
Epoch 163/200
6/6 [==============================] - 0s 2ms/step - loss: 0.1464 - accuracy: 0.9607
Epoch 164/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0951 - accuracy: 0.9438
Epoch 165/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1450 - accuracy: 0.9438
Epoch 166/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1295 - accuracy: 0.9551
Epoch 167/200
6/6 [=====

In [6]:
def clean_text(text): 
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text, vocab): 
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens: 
        for idx, word in enumerate(vocab):
            if word == w: 
                bow[idx] = 1
    return np.array(bow)

def pred_class(text, vocab, labels): 
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.2
    y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_response(intents_list, intents_json): 
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents: 
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

In [ ]:
from flask import Flask, render_template, request, redirect, url_for 

app = Flask(__name__)
app.static_folder = 'static'

r = sr.Recognizer()

@app.route("/")
def home():
    return render_template("index.html")
    
@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    intents = pred_class(userText, words, classes)
    return str(get_response(intents, data))

@app.route('/result',methods = ['POST', 'GET'])
def result():
    if request.method == 'POST':
        result = request.form
        return render_template("result.html",result = result)
    
if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jul/2021 10:56:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 10:56:36] "GET /get?msg=bsie HTTP/1.1" 200 -
